In [1]:
load('../sim_data/block_compLasso.RData')

In [2]:
dir = '../sim_data'
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

rou.list = seq(0.1, 0.9, 0.2)

files = NULL
for (rou in rou.list){
  for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_block_corr', rou, paste('P', p, 'N', n, sep='_'), '.RData', sep=''))
  }
}

In [3]:
avg_FDR = NULL
for (i in 1:length(files)){
    sim_file = files[i]
    load(sim_file, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_block_compLasso[[i]]$Stab.table
    
    FDR = NULL
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}

In [4]:
table_block = NULL
tmp_num_select = rep(0, length(results_block_compLasso))
for (i in 1:length(results_block_compLasso)){
    table_block = rbind(table_block, results_block_compLasso[[i]][c('n', 'p', 'rou', 'FP', 'FN', 'MSE', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_block_compLasso[[i]]$Stab.table))
    
}
table_block = as.data.frame(table_block)
table_block$num_select = tmp_num_select
table_block$FDR = round(avg_FDR,2)

In [6]:
head(table_block)

n,p,rou,FP,FN,MSE,Stab,num_select,FDR
50,50,0.1,2.8 ( 0.36 ),5.01 ( 0.09 ),0.36 ( 0.01 ),0.03,3.79,0.62
100,50,0.1,4.96 ( 0.55 ),3.77 ( 0.14 ),0.32 ( 0.01 ),0.1,7.19,0.55
500,50,0.1,8.81 ( 0.4 ),0.78 ( 0.08 ),0.28 ( 0 ),0.26,14.03,0.60
1000,50,0.1,8.37 ( 0.37 ),0.24 ( 0.06 ),0.27 ( 0 ),0.31,14.13,0.56
50,100,0.1,2.54 ( 0.37 ),5.22 ( 0.08 ),0.37 ( 0.02 ),0.04,3.32,0.63
100,100,0.1,3.99 ( 0.57 ),4.63 ( 0.1 ),0.35 ( 0.01 ),0.07,5.36,0.56


In [7]:
# export result
result.table_block <- apply(table_block,2,as.character)
rownames(result.table_block) = rownames(table_block)
result.table_block = as.data.frame(result.table_block)

# extract numbers only for 'n' & 'p'
result.table_block$n = tidyr::extract_numeric(result.table_block$n)
result.table_block$p = tidyr::extract_numeric(result.table_block$p)
result.table_block$ratio = result.table_block$p / result.table_block$n

result.table_block = result.table_block[c('n', 'p', 'rou', 'ratio', 'Stab', 'MSE', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_block)[1:4] = c('N', 'P', 'Corr', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead
extract_numeric() is deprecated: please use readr::parse_number() instead


In [8]:
# convert interested measurements to be numeric
result.table_block$Stab = as.numeric(as.character(result.table_block$Stab))
result.table_block$MSE_mean = as.numeric(substr(result.table_block$MSE, start=1, stop=4))
result.table_block$FP_mean = as.numeric(substr(result.table_block$FP, start=1, stop=4))
result.table_block$FN_mean = as.numeric(substr(result.table_block$FN, start=1, stop=4))
result.table_block$FN_mean[is.na(result.table_block$FN_mean)] = 0
result.table_block$num_select = as.numeric(as.character(result.table_block$num_select))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [9]:
# check whether missing values exists
result.table_block[rowSums(is.na(result.table_block)) > 0,]

,N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
21,50,100,0.3,2.0,0.27,0.94 ( 0.05 ),6 ( 0.34 ),1.93 ( 0.14 ),10.07,0.54,0.94,NA,1.93
26,100,500,0.3,5.0,0.30,0.73 ( 0.03 ),9 ( 0.76 ),0.99 ( 0.12 ),14.01,0.52,0.73,NA,0.99
51,500,50,0.7,0.1,0.90,1 ( 0.04 ),0.58 ( 0.19 ),0 ( 0 ),6.58,0.06,NA,0.58,0.00


In [10]:
# recover values
result.table_block$FP_mean[is.na(result.table_block$FP_mean)] = 6
result.table_block$MSE_mean[is.na(result.table_block$MSE_mean)] = 1

In [11]:
result.table_block[c(21,26,51), ]

,N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
21,50,100,0.3,2.0,0.27,0.94 ( 0.05 ),6 ( 0.34 ),1.93 ( 0.14 ),10.07,0.54,0.94,6.00,1.93
26,100,500,0.3,5.0,0.30,0.73 ( 0.03 ),9 ( 0.76 ),0.99 ( 0.12 ),14.01,0.52,0.73,6.00,0.99
51,500,50,0.7,0.1,0.90,1 ( 0.04 ),0.58 ( 0.19 ),0 ( 0 ),6.58,0.06,1.00,0.58,0.00


In [12]:
head(result.table_block)

N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
50,50,0.1,1.00,0.03,0.36 ( 0.01 ),2.8 ( 0.36 ),5.01 ( 0.09 ),3.79,0.62,0.36,2.80,5.01
100,50,0.1,0.50,0.10,0.32 ( 0.01 ),4.96 ( 0.55 ),3.77 ( 0.14 ),7.19,0.55,0.32,4.96,3.77
500,50,0.1,0.10,0.26,0.28 ( 0 ),8.81 ( 0.4 ),0.78 ( 0.08 ),14.03,0.6,0.28,8.81,0.78
1000,50,0.1,0.05,0.31,0.27 ( 0 ),8.37 ( 0.37 ),0.24 ( 0.06 ),14.13,0.56,0.27,8.37,0.24
50,100,0.1,2.00,0.04,0.37 ( 0.02 ),2.54 ( 0.37 ),5.22 ( 0.08 ),3.32,0.63,0.37,2.54,5.22
100,100,0.1,1.00,0.07,0.35 ( 0.01 ),3.99 ( 0.57 ),4.63 ( 0.1 ),5.36,0.56,0.35,3.99,4.63


In [13]:
tail(result.table_block)

,N,P,Corr,Ratio,Stab,MSE,FP,FN,num_select,FDR,MSE_mean,FP_mean,FN_mean
75,500,500,0.9,1.0,0.93,1.27 ( 0.05 ),0.44 ( 0.18 ),0 ( 0 ),6.44,0.04,1.27,0.44,0.00
76,1000,500,0.9,0.5,0.96,1.31 ( 0.05 ),0.23 ( 0.06 ),0 ( 0 ),6.23,0.03,1.31,0.23,0.00
77,50,1000,0.9,20.0,0.23,3.37 ( 0.19 ),5.29 ( 0.41 ),3.13 ( 0.15 ),8.16,0.57,3.37,5.29,3.13
78,100,1000,0.9,10.0,0.56,1.14 ( 0.05 ),4.56 ( 0.38 ),0.07 ( 0.03 ),10.49,0.37,1.14,4.56,0.07
79,500,1000,0.9,2.0,0.96,1.28 ( 0.04 ),0.27 ( 0.11 ),0 ( 0 ),6.27,0.03,1.28,0.27,0.00
80,1000,1000,0.9,1.0,0.95,1.35 ( 0.05 ),0.29 ( 0.09 ),0 ( 0 ),6.29,0.03,1.35,0.29,0.00


In [14]:
## export
write.table(result.table_block, '../results_summary/sim_block_compLasso.txt', sep='\t', row.names=F)

In [10]:
library(ggplot2)
library(ggpubr)
result.table_block$N = as.factor(result.table_block$N)
fig_block_stab = ggplot(result.table_block, aes(x=P, y=Stab, color=N)) + 
               geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position = "none") +
               scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('Stability')

fig_block_mse = ggplot(result.table_block, aes(x=P, y=MSE_mean, color=N)) + 
              geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position="none") +
              scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('MSE')

fig_block_fp = ggplot(result.table_block, aes(x=P, y=FP_mean, color=N)) + 
             geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position = "none") +
             scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('False Positives')

fig_block_fn = ggplot(result.table_block, aes(x=P, y=FN_mean, color=N)) + 
             geom_point(aes(size = Corr, alpha=Corr)) + theme(legend.position = "none") +
             scale_size_discrete(range = c(1,4)) + scale_alpha_discrete(range = c(1, 0.4)) + ylab('False Negatives')
fig = ggarrange(fig_block_stab, fig_block_mse, fig_block_fp, fig_block_fn, ncol=2, nrow=2, 
          common.legend = TRUE, legend="right") 
fig = annotate_figure(fig, top = text_grob("Block_compLasso"))
ggexport(fig, filename = "../figures_sim/figure_block_compLasso.pdf", height=6, width=6)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Loading required package: magrittr
Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”Warning message:
“Using size for a discrete variable is not advised.”Warning message:
“Using alpha for a discrete variable is not advised.”file saved to ../figures_sim/figure_block_compLasso.pdf


In [11]:
result.table_block[with(result.table_block, order(N, P, Corr)),]

,N,P,Corr,Ratio,Stab,MSE,FP,FN,MSE_mean,FP_mean,FN_mean
1,50,50,0.1,1.0,0.03,0.36 ( 0.01 ),2.8 ( 0.36 ),5.01 ( 0.09 ),0.36,2.80,5.01
17,50,50,0.3,1.0,0.32,0.8 ( 0.05 ),5.34 ( 0.32 ),1.21 ( 0.12 ),0.80,5.34,1.21
33,50,50,0.5,1.0,0.42,0.77 ( 0.04 ),5.49 ( 0.31 ),0.25 ( 0.06 ),0.77,5.49,0.25
49,50,50,0.7,1.0,0.49,0.8 ( 0.06 ),4.27 ( 0.31 ),0.24 ( 0.06 ),0.80,4.27,0.24
65,50,50,0.9,1.0,0.48,0.92 ( 0.07 ),4.7 ( 0.32 ),0.14 ( 0.04 ),0.92,4.70,0.14
5,50,100,0.1,2.0,0.04,0.37 ( 0.02 ),2.54 ( 0.37 ),5.22 ( 0.08 ),0.37,2.54,5.22
21,50,100,0.3,2.0,0.27,0.94 ( 0.05 ),6 ( 0.34 ),1.93 ( 0.14 ),0.94,6.00,1.93
37,50,100,0.5,2.0,0.36,1.03 ( 0.07 ),5.95 ( 0.33 ),0.96 ( 0.11 ),1.03,5.95,0.96
53,50,100,0.7,2.0,0.42,1.14 ( 0.08 ),5.46 ( 0.32 ),0.55 ( 0.09 ),1.14,5.46,0.55
69,50,100,0.9,2.0,0.42,1.28 ( 0.08 ),5.86 ( 0.29 ),0.47 ( 0.09 ),1.28,5.86,0.47
